#### In this little notebook we show how to access the relational_lsplines code.

* First let's import the basic library

In [25]:
import relational_lsplines as rlspline

* And we want the rules tiny interface language and miniKanren-esque logic programming stuff

In [26]:
lp = rlspline.lp # miniKanren - like relational logic, extended for intervals.
ia = rlspline.ia # interval arithmetic class

* Next let's get access to the search code

In [27]:
from relational_lsplines.simple_hull_rules_language import HullGeometryGenerator

* Make some design variables.  In our tiny language, these are called PStates objects.

In [28]:
"""Make some design space variables:
"""
lwl = lp.PStates(name='lwl')
bwl = lp.PStates(name='bwl')
draft = lp.PStates(name='draft')
vol = lp.PStates(name='vol')
disp = lp.PStates(name='disp')

Cb = lp.PStates(name='Cb')
Cp = lp.PStates(name='Cp')


* Make some basic relationships.  These relationships between variables can represent various things.
* For example, we could simply say that one variable is equivalent to another:

In [29]:
disp = disp == vol

* Or, as another example, we could write down some relationship from naval architecture that must hold:

In [30]:
"""-----------------------------------------------
#rule: block coefficient
"""
Cb = Cb == vol/(lwl*bwl*draft)

* The slightly odd syntax is there because we are using an internal domain specific language.
* It says $C_b$ is equal to $vol$ divided by $lwl \times bwl \times draft$
* I am going to skip out of line here and tell you that this computation runs... "all directions".
This is what it means to say that we are programming "relationally"
* We are following in the footsteps of the logic programming language Prolog and especially the relational logic programming langauge miniKanren, but with a focus on intervals. (It's a long story)
* This tiny language includes a declarative logic programming engine which understands interval arithmetic.
* It uses python to parse itsef, which really makes things easy and fun to work with!  (trust me)
* I think it's fun to look at lthe computation tree self assembled by the language when we write these rules.

For example, the computation tree for this funny looking, but simple, rule from above:


   $C_{b} = C_{b} == vol/(lwl*bwl*draft)$ 


Looks like this:


In [31]:
print Cb

'Cb'==
   'v9'/
      'vol' 
        
      'v8'*
         'v7'*
            'lwl' 
        
            'bwl' 
        
         'draft' 
        



* As you might see, the printed contents of $C_{b}$ are a tree.  $C_{b}$ is what I call the 'top node' of this tree.  It's also the node which most clearly encapsulates the rule, loosely speaking. 

* When we created the rule above, the tiny language created this computational graph for us, using python internally to figure out the parsing.

### PStates objects parse themselves to build up computational graphs. 

* Another part of the tiny language can read this graph and construct a list of rules for us that will make the design space honor the relationship we've just spelled out here.  (We will access it with a function called "set_rules" in just a little bit)

* It might be worth noting that the extra data you see in the tree - that is, the 'number-names' - those parts of the tree, symbols $v16$, $v17$, and $v18$, which do not match up with variables we've input above, are actually placeholders which store partial evaluations of the rule for us.  That is, when any computer programming language has to evaluate something like a mathematical expression:


In [32]:
a = 1
b = 2
c = 3
d = a*(a+b+c)

* Underneath the hood, the langauge turns this expression into a graph that is similar to the way my graph was constructed above.  We don't usually see it, or care, but the machine breaks the computation into bite size chuncks, with nodes in a graph at each chunck - the intersection of two atoms combined with some operation (say +) to then be rolled up into further computations.

* All this intermmediate data is what we want.  By getting access to this kind of fine grained information about  mathematical operations, we can parse it down into a bunch of interrelated atomistic computations, and.... the point of it all is that by defining how those basic atomistic computations work, we can make them do whatever we want.  -Including turning them into relational constraint networks.
[comment]: <> (programs a la miniKanren, and hooking them together automatically into constraint networks.  (In fact, only that last bit - the constraint network, building it automatically - only that part requires the special parsing on display here.  In another Jupyter notebook, we will show the miniKanren part without use of the parse tree)
* And when we want to build up rules nets, what we want is to turn those regular old, vanilla arithmetic expressions into declarative relations.  

* But when we build up declarative relations, we need to store the current best known answers for each fine grained, atomistic, node of data in the graph.  

* This is way more data than you want to keep track of by hand.  We have to store, separately, specially, every combination of two variables into a third for every sub-component of every calculation.

* And, we can't really let there be any naming conflicts!


* No fun.


* Oh wait, we have the computational graph...  Just make the computer do it.

### So that's why we store the computational graph in the example above.

* We store it in the variables themselves, because, well, it's dead simple with operator overloading.

* Great.

* What do we do with $C_{b}$ now?

In [33]:
#get a helper class for the langugage: (this is going to hide good stuff though)
from relational_lsplines.simple_hull_rules_language import HullGeometryGenerator

#launch it
hullrulesnet = HullGeometryGenerator(rtype='gauss',
                                     verbose=True) 


hullrulesnet.set_rules(Cb,disp)

* In the code snippet above, we added out rule, $C_{b}$ to a new rules database we call $hullrulesnet$.

* Oh!  we also added $disp$ - don't forget we added that rule too!

* I wonder what $C_b$ looks like if we print it now?

In [34]:
print Cb

'Cb'==



Whoa!

* Our "interesting" little graph is gone.  What gives?

* Well, as a hint, the aim of this project is to put a bunch of rules together in order to reason in a nice way about ship hull form geometry.  
   *   There are going to be a lot of rules
   *   Some of them are going to be using the same symbols in different ways.  (Turns out this is a very good thing.  One might say it's the point, really)

* Sooo, if a symbol (PStates variable) gets used in more than one rule, what does the graph look like?

* Well, you could think of tacking one graph onto the other, just as if you written the two rules as one stuck together at an equal sign somewhere...

* But what if you have 2 or more symbols in both rules that's a lot of cross talk.  Things are not going to make sense.

### To ensure this is not a problem, a couple of things have to happen:
1. When you enter a rule into the rules network, the computational graph must automatically get scrubbed.  This happens automatically when you drop the rule-node into the "set_rules" function

2. The user must promise never to mix the same variables in two complicated rules at once.  In practice it's easy - if your rule involves several variables, add it to the rules base right away!

### So you must remember only one thing:  don't reuse variables before setting the rules.

In [35]:
## Always set your rules!:
#
# set rules using a function like this:
#
""">> hullrulesnet.set_rules(Cb,disp)"""
# we already did this above so I will not set it again.
# if we did we would get a second rule the same as the first.  That would be inefficient.

'>> hullrulesnet.set_rules(Cb,disp)'

* Oh shoot, one more thing must be remembered.  Namely that only $+-*/ <= $ are presently supported in the tiny language

### Only $+-*/ <=$ are supported so far.  (Don't worry this is enough to "do something')


* That's it for the gotchas.  Any rule should be fair game as long as it is composed with add subtract multiply and divide.  $<=$ is a less than rule.  I can add $>=$ too but there has really been no need so far.

* Sorry that this "hullrulesnet" stuff is really "high level" - sometimes it's nice to work with the language more directly.  Another notebook will demonstate more language features including some of what "hullrulesnet" is doing in some detail.

*  What's that?  Perhaps you want to actually see the rules in action right now?  Okay let's do it.  Then will call this one done.

* Are you familiar with interval arithmetic?  See the wikipedia article on it for a quick refresher if not.  A notebook will be coming soon.

* Right now it may be enough to note that an interval is a continuous line segment from the minimum, denoted by the number on the left hand side of the interval, to the maximum, denoted by the number on the right hand side of the interval.

* Computations with intervals are performed such that the answer to a calculation always encloses the set of all real numbers that could be reached by real arithmetic if the values were chosen from the intervals.

In [36]:
# once again, here is our rule:
#
#   Cb = Cb == vol/(lwl*bwl*draft)

# it lives in our "design space"
#
# which is composed of all these PStates objects:
#
#  From before:
"""
lwl = lp.PStates(name='lwl')  # length (lenght of our ship hull at the waterline)
bwl = lp.PStates(name='bwl')  # beam (width)
draft = lp.PStates(name='draft') #depth
vol = lp.PStates(name='vol') #total displacement (a measure of volume in terms of mass of water "displaced")
disp = lp.PStates(name='disp') #renaming of vol to make the engineers happy.  (if I'd only stick with it)

Cb = lp.PStates(name='Cb')  # block coefficient
Cp = lp.PStates(name='Cp')  # prismatic coefficient
"""

#
#
# start inputing data.  Just chose some intervals that kind of make sense.
#
# we are going to be totally ignoring some of the design space to start!
#
#
# Just for the heck of it I will pick some weird stuff:

# Block Coefficient - naval architectrue term composed of the fraction
# of the actual  total displacement of a ship hull over the block of volume
# this hull would sit in if the extremes of the hull were extruded into a minimally
# enclosing "block"
# - do note that this isn't far from the concept of an interval vector.
Cb = Cb == ia(-1.,1.)
#
# I am picking an interval of -1 to 1 on purpose.  This is a nonsensical choice
# but I want to show that our system can cope with it happily.
#

# water line length 
# Let's supppose we are designing a ship and we think it needs to be between 80 and 120 length units
#
# perhaps the customer has made this a constraint.  -It is in the design spec.
#
lwl = lwl == ia(80.,120.)

# Same for displacement, here represented by the total geometric volume we'd like
# our vessel to have in units of length units cubed.
vol= vol == ia(101000.,202000.)


# draft, to be somewhat specific, draft is the
#  vertical distance between the waterline 
#  and the bottom of the hull (keel), with the thickness of the hull included.
#
# Well our hull has no thickness right now.  Our keel is to be flat at the lowest point.
#
# draft is here just the vertical distance from the water line of the ship 
# down to the lowest point.
#
#
draft = draft == ia(-20.,40.)
#
# Let's make it be a really weird interval.  Just to make trouble.
#

#
# Now is the time to use that "set_rules" function alluded to above:
#
hullrulesnet.set_rules(Cb, 
                      lwl,
                      vol,
                      draft)

# it's going to automatically propogate the information 
# we've input here around the network.



splitting the interval
splitting the interval
splitting the interval
splitting the interval
splitting the interval
splitting the interval
splitting the interval
splitting the interval
splitting the interval
splitting the interval
splitting the interval
splitting the interval
splitting the interval
splitting the interval
splitting the interval
splitting the interval
splitting the interval


* Hey!  what's all this _splitting_ _the_ _interval_ stuff?
* What does our design space look like now?

In [37]:
print hullrulesnet.rgp.env

States([{
  disp:ia(101000.0,202000.0)  
  v9:ia(-1.0,-5e-26)  
  draft:ia(-20.0,-5e-51)  
  v8:ia(-2.02e+30,-101000.0)  
  Cb:ia(-1.0,-5e-26)  
  bwl:ia(42.0833333333,2.525e+53)  
  v7:ia(5050.0,2.02e+55)  
  vol:ia(101000.0,202000.0)  
  lwl:ia(80.0,120.0)  
}, {
  disp:ia(101000.0,202000.0)  
  v9:ia(-1.0,-5e-26)  
  vol:ia(101000.0,202000.0)  
  v8:ia(-2.02e+30,-101000.0)  
  bwl:ia(-2.525e+53,-21.0416666667)  
  Cb:ia(-1.0,-5e-26)  
  lwl:ia(80.0,120.0)  
  v7:ia(-2.02e+55,-2525.0)  
  draft:ia(5e-51,40.0)  
}, {
  disp:ia(101000.0,202000.0)  
  v9:ia(5e-26,1.0)  
  v8:ia(101000.0,2.02e+30)  
  v7:ia(-2.02e+55,-5050.0)  
  Cb:ia(5e-26,1.0)  
  lwl:ia(80.0,120.0)  
  bwl:ia(-2.525e+53,-42.0833333333)  
  draft:ia(-20.0,-5e-51)  
  vol:ia(101000.0,202000.0)  
}, {
  disp:ia(101000.0,202000.0)  
  v8:ia(101000.0,2.02e+30)  
  v9:ia(5e-26,1.0)  
  vol:ia(101000.0,202000.0)  
  lwl:ia(80.0,120.0)  
  bwl:ia(21.0416666667,2.525e+53)  
  Cb:ia(5e-26,1.0)  
  v7:ia(2525.0,2.02e+55)  
  dr

* Ok, it clearly did something.  (it divided by zero to be precise!)
* Furthermore, it __removed__ the zeros from the divisors of the newly split design spaces.
* What??  Yes.  And this is a really good thing.
* See how many times it split intervals?  Some of the new design spaces were infeasible from the getgo.  Those were discarded.  
* That's the only reason I put in the "weird" numbers - just to show that the network would automatically deal with them.
* The remaining spaces contain __no__ singularities.   
* Much work was accomplished by __dividing__ __by__ __zero__!
* But maybe we made our design space kind of wonky.  This is a contrived example after all.
* Let's fix it with another rule

In [38]:
Cb = Cb == ia(0.,1.)



hullrulesnet.set_rules(Cb)



In [39]:
print hullrulesnet.rgp.env

States([{
  disp:ia(101000.0,202000.0)  
  Cb:ia(5e-26,1.0)  
  v7:ia(-2.02e+55,-5050.0)  
  draft:ia(-20.0,-5e-51)  
  v9:ia(5e-26,1.0)  
  v8:ia(101000.0,2.02e+30)  
  lwl:ia(80.0,120.0)  
  bwl:ia(-2.525e+53,-42.0833333333)  
  vol:ia(101000.0,202000.0)  
}, {
  Cb:ia(5e-26,1.0)  
  v7:ia(2525.0,2.02e+55)  
  draft:ia(5e-51,40.0)  
  vol:ia(101000.0,202000.0)  
  v8:ia(101000.0,2.02e+30)  
  v9:ia(5e-26,1.0)  
  lwl:ia(80.0,120.0)  
  disp:ia(101000.0,202000.0)  
  bwl:ia(21.0416666667,2.525e+53)  
}])


* Still to crazy for me.  Draft can still be negative.  Let's get rid of that.
* Oh, since we are looking at this, you might wonder what's with all those random symbols like _v17_ and the like?  -We mentioned them above...

In [40]:
draft = draft == ia(2.,40.)

hullrulesnet.set_rules(draft)

print hullrulesnet.rgp.env

States([{
  Cb:ia(5e-26,1.0)  
  v7:ia(2525.0,1.01e+30)  
  draft:ia(2.0,40.0)  
  vol:ia(101000.0,202000.0)  
  v8:ia(101000.0,2.02e+30)  
  v9:ia(5e-26,1.0)  
  lwl:ia(80.0,120.0)  
  disp:ia(101000.0,202000.0)  
  bwl:ia(21.0416666667,1.2625e+28)  
}])


* Hey look! we got a design interval for $bwl$ even though we never put anything in!
* How did that happen?  

Well the $C_{b}$ relationship we input above sent information around the constraint network and the relation figured out that $bwl$ had to be in this range to make everything consistent.

* (It "figured this out" via relational interval arithmetic, something that will be explained elsewhere)
* Let continue to improve our bounds on $bwl$ indirectly.  

In [41]:
# a more sensible design interval for the block coefficient might be this:
Cb = Cb == ia(.7,.85)



hullrulesnet.set_rules(Cb)

print hullrulesnet.rgp.env

States([{
  v7:ia(2970.58823529,144285.714286)  
  Cb:ia(0.7,0.85)  
  draft:ia(2.0,40.0)  
  vol:ia(101000.0,202000.0)  
  v8:ia(118823.529412,288571.428571)  
  v9:ia(0.7,0.85)  
  disp:ia(101000.0,202000.0)  
  lwl:ia(80.0,120.0)  
  bwl:ia(24.7549019608,1803.57142857)  
}])


In [42]:
# and draft maybe something like this:

draft = draft == ia(20.,40.)



hullrulesnet.set_rules(draft)

print hullrulesnet.rgp.env

States([{
  v7:ia(2970.58823529,14428.5714286)  
  draft:ia(20.0,40.0)  
  vol:ia(101000.0,202000.0)  
  v8:ia(118823.529412,288571.428571)  
  Cb:ia(0.7,0.85)  
  v9:ia(0.7,0.85)  
  lwl:ia(80.0,120.0)  
  disp:ia(101000.0,202000.0)  
  bwl:ia(24.7549019608,180.357142857)  
}])


* Well, this example isn't perfect.  
* But you can get a sense how it goes
* We input what we know and the relational constraints enforce what we input and figure out what else can be deduced from it.
* The design space is only ever narrowed (splitting only narrows as well) once it becomes interval valued.  (This is really important - to be elucidated elsewhere!)
* The part where we split the designspace around the division by zero was a bit convoluted and complicated.  Sorry about that.

* Wait one second!  What about the "random" symbols like "v16" and "v17" again?
* The answer is that those are conective variables that the language is managing.
* Those store partial results of our rules and connect one "atomistic relational expression" to another (or several).  In this way we are free to compose rules with some freedom from the underlying relational logic.
* Without them we would have to construct connectives for partial results by hand -  in order to run the rules in an iterative network (for convergence... idemoptence... I am digressing).
* This would not be very fun.  In fact, I would say this would limit the applicability of our system to "small models" and "small experimental code".
* Luckily, this doesn't have to be the case.